In [184]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder


In [185]:
#Importamos los datos
df = pd.read_csv("../data/NCDB_1999_to_2014.csv")

C:\Users\Usuario\anaconda3\envs\core_models\lib\site-packages\IPython\core\interactiveshell.py:3417: DtypeWarning: Columns (1,2,5,12) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [186]:
# Cambio de nombre de las variables
new_names = ["year", "month", "weekday", "hour", "fatality", "vehicles_involved", "crash_type", "crash_place", "crash_weather",
"surface_state", "road_slope", "traffic_state", "vehicle_id", "vehicle_type", "vehicle_year", "passenger_id", "passenger_sex",
"passenger_age", "passenger_role", "passenger_fatality", "passenger_safety", "passenger_type"]

df.columns = new_names

In [187]:
# El dataset contiene variables que no están disponible en el momento de la colisión y por tanto escapan al objetivo de nuestro ánalisis y
# otras que directamente no son de útilidad como IDs. También debemos prescindir de registros duplicados antes de eliminar los IDs
df = df.drop_duplicates().drop(["passenger_id","passenger_fatality"], axis=1)

In [188]:
# Recodeado de la variable objetivo
# Antes: 1-fatality 2-no fatality
# Después: 1-fatality 0-no fatality
df['fatality'] = df['fatality'].replace({2:0})

In [189]:
#Ponemos la variable objetivo a la derecha del dataframe
new_columns = list(df.columns[0:4]) + list(df.columns[5:]) + list(df.columns[4:5])
df = df[new_columns]

In [190]:
# Remplazamos U, UU y UUUU por NAs
df = df.replace({"U": np.nan, "UU": np.nan, "UUUU": np.nan})

In [191]:
#Eliminando los registros con >40% nulos
null_rows = df.isnull().sum(axis=1).sort_values(ascending=False)
nulos_filas = pd.DataFrame(null_rows, columns=['nulos_filas'])  
nulos_filas['target'] = df['fatality'].copy()
nulos_filas['porcentaje_filas']= nulos_filas['nulos_filas']/df.shape[1]
print(nulos_filas)
nulos_40 = list(nulos_filas.index[nulos_filas.porcentaje_filas>=0.40])
df = df.loc[set(df.index)-set(nulos_40)].reset_index()

         nulos_filas  target  porcentaje_filas
5245712           16       1              0.80
5245711           16       1              0.80
2884064           13       0              0.65
769190            13       0              0.65
1244965           12       0              0.60
...              ...     ...               ...
2665912            0       0              0.00
2665913            0       0              0.00
2665914            0       0              0.00
2665915            0       0              0.00
2930693            0       0              0.00

[5855336 rows x 3 columns]


In [192]:
# Cambio de la variable vehicle_year a años de antigüedad y eliminamos vehicle_year
# Cambiamos passenger_age y vehicles_involved a númericas ninguna de las dos tiene NAs. 
# Sin embargo, passenger_age tiene valores NN para transformar la variable los haremos NA con 'coerce'
df['vehicle_year'] = pd.to_numeric(df.year - pd.to_numeric(df.vehicle_year, errors= "coerce"))
df = df.rename(columns={"vehicle_year": 'vehicle_age'})
df['passenger_age'] = pd.to_numeric(df['passenger_age'],"coerce")
df['vehicles_involved'] = pd.to_numeric(df['vehicles_involved'],"ignore")
df.drop(inplace=True, axis=1, columns='index')

In [193]:
# Month a numerico. Limpieza de valores 0
df["month"] = np.int8(df["month"].replace({"01": 1, "02": 2, "11": 11, "12" : 12}))
df = df[df["month"] != 0]

In [194]:
# Weekday a numerico. Limpieza de valores 0
df["weekday"] = np.int8(df["weekday"].replace({"7": 7, "1": 1, "2": 2, "3" : 3, "4":4, "5":5, "6":6}))
df = df[df["weekday"] != 0]

In [195]:
# Hour a numerico. Limpieza de valores nulos
df = df[df["hour"].notnull()]
df["hour"] = df["hour"].astype("int8")

In [196]:
# Replace M/F to 1/0
df.passenger_sex.replace({"M":1, "F":0}, inplace=True)

# Remove NA values (15k rows, 0.3% of total sample)
df.passenger_sex.replace('[^0-9]+',np.nan,regex=True,inplace=True)
df.passenger_sex.dropna(inplace=True)

In [197]:
# Drop passenger_safety==11, very low sample
df = df.loc[df['passenger_safety'] != "11"]

In [198]:
# Remove rows with an outlier vehicle_age
df = df.loc[df['vehicle_age'] < 30]

# Removes rows with outlier vehicles_involved
df = df.loc[df['vehicles_involved'] < 6]

In [199]:
#Transform month column to quarters
df['month'] = (df['month']-1)//3 + 1
df = df.rename({'month': 'quarter'}, axis= 1)

In [200]:
# Transform weekday column
df['weekday'] = df['weekday'].replace({6:3, 7:3, 1:2, 4:2, 5:2, 2:1, 3:1})

In [201]:
# Transform hour column
df['hour'] = df['hour'].replace({0:1, 1:1, 2:1, 3:1, 4:1, 5:1, 6:2, 7:2, 8:2, 9:3, 10:3, 11:3, 11:4, 12:4, 13:4, 14:4, 15:4, 16:4, 17:4, 18:4, 19:5, 20:5, 21:5, 22:6, 23:6})

In [202]:
df

,year,quarter,weekday,hour,vehicles_involved,crash_type,crash_place,crash_weather,surface_state,road_slope,traffic_state,vehicle_id,vehicle_type,vehicle_age,passenger_sex,passenger_age,passenger_role,passenger_safety,passenger_type,fatality
0,1999,1,2,5,2.0,34,NaN,1,5,3,03,01,06,9.0,1,41.0,11,NaN,1,0
1,1999,1,2,5,2.0,34,NaN,1,5,3,03,02,01,12.0,1,19.0,11,NaN,1,0
2,1999,1,2,5,2.0,34,NaN,1,5,3,03,02,01,12.0,0,20.0,13,02,2,0
3,1999,1,2,2,1.0,01,NaN,5,3,6,18,01,01,13.0,1,46.0,11,NaN,1,0
5,1999,1,2,4,3.0,QQ,QQ,1,2,1,01,01,01,15.0,1,28.0,11,NaN,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5850156,2014,4,3,4,2.0,41,01,1,5,1,18,2,05,6.0,1,44.0,11,02,1,0
5850157,2014,4,3,4,2.0,41,01,1,5,1,18,2,05,6.0,1,34.0,13,02,2,0
5850158,2014,4,3,5,1.0,03,01,1,3,4,18,1,01,13.0,0,35.0,11,02,1,0
5850159,2014,4,3,5,1.0,03,01,1,3,4,18,1,01,13.0,1,26.0,13,02,2,0
